In [2]:
import requests
from datetime import datetime
import pandas as pd
import xml.etree.ElementTree as ET

# Replace with your actual Client ID and API Key
client_id = 'bef6017966eb3a7883800d136b0e8de4'
api_key = 'a7dfa9a925d50c6b8cb226cd3405b2f3'



In [2]:
# API endpoint for real-time changes at Frankfurt(Main) Hbf (EVA number 8000105)
url = 'https://apis.deutschebahn.com/db-api-marketplace/apis/timetables/v1/fchg/8000105'

# Headers for authentication
headers = {
    'DB-Client-Id': client_id,
    'DB-Api-Key': api_key
}

# Make the API request
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Parse the XML response
    root = ET.fromstring(response.content)
    
    # Extract data from XML
    data_list = []
    
    for s in root.findall('.//s'):
        # Extract station, arrivals, and departures
        station_name = root.attrib.get('station', 'N/A')
        
        # Extract arrival data
        for ar in s.findall('ar'):
            planned_arrival = ar.attrib.get('pt', None)
            actual_arrival = ar.attrib.get('ct', None)
            planned_path = ar.attrib.get('ppth', 'N/A')  # Planned path for the arrival
            
            # Append only if both planned and actual arrivals exist
            if planned_arrival or actual_arrival:
                data_list.append({
                    'station': station_name,
                    'planned_path': planned_path,
                    'planned_arrival_time': planned_arrival,
                    'actual_arrival_time': actual_arrival,
                    'planned_departure_time': None,
                    'actual_departure_time': None
                })
        
        # Extract departure data
        for dp in s.findall('dp'):
            planned_departure = dp.attrib.get('pt', None)
            actual_departure = dp.attrib.get('ct', None)
            planned_path = dp.attrib.get('ppth', 'N/A')  # Planned path for the departure
            
            # Append only if both planned and actual departures exist
            if planned_departure or actual_departure:
                data_list.append({
                    'station': station_name,
                    'planned_path': planned_path,
                    'planned_arrival_time': None,
                    'actual_arrival_time': None,
                    'planned_departure_time': planned_departure,
                    'actual_departure_time': actual_departure
                })
    
    # Convert to DataFrame for easier handling
    df = pd.DataFrame(data_list)
    
    # Replace empty strings or None with 'N/A'
    df.fillna('N/A', inplace=True)
    
    # Display the DataFrame
    print(df)

    # Save the data to a CSV file for future analysis
    df.to_csv('train_realtime_changes.csv', index=False)
    print("Data saved to 'train_realtime_changes.csv'")
else:
    print(f"Error {response.status_code}: {response.text}")


                station                                       planned_path  \
0    Frankfurt(Main)Hbf                                                N/A   
1    Frankfurt(Main)Hbf                                                N/A   
2    Frankfurt(Main)Hbf                                                N/A   
3    Frankfurt(Main)Hbf                                                N/A   
4    Frankfurt(Main)Hbf                                                N/A   
..                  ...                                                ...   
755  Frankfurt(Main)Hbf                                                N/A   
756  Frankfurt(Main)Hbf                                                N/A   
757  Frankfurt(Main)Hbf  Lorchhausen|Kaub|St Goarshausen|Kestert|Kamp-B...   
758  Frankfurt(Main)Hbf  München Hbf|Nürnberg Hbf|Würzburg Hbf|Aschaffe...   
759  Frankfurt(Main)Hbf  Frankfurt(M) Flughafen Fernbf|Limburg Süd|Mont...   

    planned_arrival_time actual_arrival_time planned_departure_

In [3]:
df.head()

,station,planned_path,planned_arrival_time,actual_arrival_time,planned_departure_time,actual_departure_time
0,Frankfurt(Main)Hbf,N/A,N/A,2410232016,N/A,N/A
1,Frankfurt(Main)Hbf,N/A,N/A,N/A,N/A,2410232025
2,Frankfurt(Main)Hbf,N/A,N/A,2410231715,N/A,N/A
3,Frankfurt(Main)Hbf,N/A,N/A,2410231718,N/A,N/A
4,Frankfurt(Main)Hbf,N/A,N/A,2410232004,N/A,N/A


In [24]:
# Function to fetch real-time changes for a station by EVA number
def get_real_time_changes(evaNo):
    # API endpoint for real-time changes at a specific station
    url = f'https://apis.deutschebahn.com/db-api-marketplace/apis/timetables/v1/fchg/{8000105}'

    # Headers for authentication
    headers = {
        'DB-Client-Id': client_id,
        'DB-Api-Key': api_key
    }

    # Make the API request
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the XML response
        root = ET.fromstring(response.content)
        
        # Extract data from XML
        data_list = []
        
        for s in root.findall('.//s'):
            # Extract the EVA number and service ID (if available)
            station_eva = s.attrib.get('eva', evaNo)
            station_name = f"Station EVA {station_eva}"  # Placeholder for station name
            train_service_id = s.attrib.get('id', 'N/A')

            # Extract arrival data
            for ar in s.findall('ar'):
                actual_arrival = ar.attrib.get('ct', 'N/A')
                planned_path = ar.attrib.get('cpth', 'N/A')  # Planned path for the arrival
                arrival_delay = ar.attrib.get('l', 'N/A')
                
                # Extract message details (if any) associated with the arrival
                arrival_messages = []
                for m in ar.findall('m'):
                    arrival_messages.append({
                        'message_id': m.attrib.get('id', 'N/A'),
                        'type': m.attrib.get('t', 'N/A'),
                        'code': m.attrib.get('c', 'N/A'),
                        'timestamp': m.attrib.get('ts-tts', 'N/A')  # Human-readable timestamp
                    })

                # Append data to the list
                data_list.append({
                    'station': station_name,
                    'train_service_id': train_service_id,
                    'planned_path': planned_path,
                    'planned_arrival_time': None,  # Assuming not available in this case
                    'actual_arrival_time': actual_arrival,
                    'planned_departure_time': None,
                    'actual_departure_time': None,
                    'arrival_delay': arrival_delay,
                    'departure_delay': 'N/A',  # No departure info in this section
                    'cancellation_flag': 'N/A',  # Handle cancellation in a separate section
                    'messages': arrival_messages if arrival_messages else 'No messages'
                })
            
            # Extract departure data
            for dp in s.findall('dp'):
                actual_departure = dp.attrib.get('ct', 'N/A')
                planned_path = dp.attrib.get('cpth', 'N/A')  # Planned path for the departure
                departure_delay = dp.attrib.get('l', 'N/A')

                # Extract message details (if any) associated with the departure
                departure_messages = []
                for m in dp.findall('m'):
                    departure_messages.append({
                        'message_id': m.attrib.get('id', 'N/A'),
                        'type': m.attrib.get('t', 'N/A'),
                        'code': m.attrib.get('c', 'N/A'),
                        'timestamp': m.attrib.get('ts-tts', 'N/A')  # Human-readable timestamp
                    })

                # Append data to the list
                data_list.append({
                    'station': station_name,
                    'train_service_id': train_service_id,
                    'planned_path': planned_path,
                    'planned_arrival_time': None,
                    'actual_arrival_time': None,
                    'planned_departure_time': None,  # Assuming not available in this case
                    'actual_departure_time': actual_departure,
                    'arrival_delay': 'N/A',  # No arrival info in this section
                    'departure_delay': departure_delay,
                    'cancellation_flag': 'N/A',  # Handle cancellation in a separate section
                    'messages': departure_messages if departure_messages else 'No messages'
                })
        
        # Check for cancellation information
        for m in root.findall('.//m'):
            if m.attrib.get('del', '0') == '1':  # If a cancellation flag exists
                data_list.append({
                    'station': station_name,
                    'train_service_id': m.attrib.get('id', 'N/A'),
                    'planned_path': 'N/A',
                    'planned_arrival_time': 'N/A',
                    'actual_arrival_time': 'N/A',
                    'planned_departure_time': 'N/A',
                    'actual_departure_time': 'N/A',
                    'arrival_delay': 'N/A',
                    'departure_delay': 'N/A',
                    'cancellation_flag': 'Yes',
                    'messages': [{
                        'message_id': m.attrib.get('id', 'N/A'),
                        'type': m.attrib.get('t', 'N/A'),
                        'timestamp': m.attrib.get('ts-tts', 'N/A')
                    }]
                })

        # Convert to DataFrame for easier handling
        df = pd.DataFrame(data_list, columns=[
            'station', 'train_service_id', 'planned_path', 'planned_arrival_time', 'actual_arrival_time',
            'planned_departure_time', 'actual_departure_time', 'arrival_delay', 'departure_delay',
            'cancellation_flag', 'messages'
        ])
        
        # Replace empty strings or None with 'N/A'
        df.fillna('N/A', inplace=True)
        
        # Display the DataFrame
        display(df)

        # Save the data to a CSV file for future analysis
        df.to_csv(f'train_realtime_changes_{evaNo}.csv', index=False)
        print(f"Data saved to 'train_realtime_changes_{evaNo}.csv'")
    else:
        print(f"Error {response.status_code}: {response.text}")

# Example EVA number for Frankfurt(Main) Hbf
eva_number = '8000105'

# Call the function
get_real_time_changes(eva_number)


,station,train_service_id,planned_path,planned_arrival_time,actual_arrival_time,planned_departure_time,actual_departure_time,arrival_delay,departure_delay,cancellation_flag,messages
0,Station EVA 8000105,9191945868145213951-2410241126-27,N/A,N/A,2410241500,N/A,N/A,54,N/A,N/A,"[{'message_id': 'r8500127', 'type': 'f', 'code..."
1,Station EVA 8000105,3704713885565394549-2410241036-11,N/A,N/A,2410241416,N/A,N/A,N/A,N/A,N/A,"[{'message_id': 'r8523678', 'type': 'q', 'code..."
2,Station EVA 8000105,3704713885565394549-2410241036-11,N/A,N/A,N/A,N/A,2410241421,N/A,N/A,N/A,"[{'message_id': 'r8523678', 'type': 'q', 'code..."
3,Station EVA 8000105,-8065221203242695110-2410241304-9,N/A,N/A,2410241343,N/A,N/A,RB58,N/A,N/A,No messages
4,Station EVA 8000105,5656583513138578556-2410241010-10,N/A,N/A,2410241441,N/A,N/A,N/A,N/A,N/A,"[{'message_id': 'r8508380', 'type': 'f', 'code..."
...,...,...,...,...,...,...,...,...,...,...,...
866,Station EVA 8000105,r2260946,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Yes,"[{'message_id': 'r2260946', 'type': 'h', 'time..."
867,Station EVA 8000105,r2260946,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Yes,"[{'message_id': 'r2260946', 'type': 'h', 'time..."
868,Station EVA 8000105,r2260946,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Yes,"[{'message_id': 'r2260946', 'type': 'h', 'time..."
869,Station EVA 8000105,r2260946,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Yes,"[{'message_id': 'r2260946', 'type': 'h', 'time..."


Data saved to 'train_realtime_changes_8000105.csv'


In [25]:
import pandas as pd 
df = pd.read_csv('train_realtime_changes_8000105.csv')

In [26]:
df.head()

,station,train_service_id,planned_path,planned_arrival_time,actual_arrival_time,planned_departure_time,actual_departure_time,arrival_delay,departure_delay,cancellation_flag,messages
0,Station EVA 8000105,9191945868145213951-2410241126-27,NaN,NaN,2.410242e+09,NaN,NaN,54,NaN,NaN,"[{'message_id': 'r8500127', 'type': 'f', 'code..."
1,Station EVA 8000105,3704713885565394549-2410241036-11,NaN,NaN,2.410241e+09,NaN,NaN,NaN,NaN,NaN,"[{'message_id': 'r8523678', 'type': 'q', 'code..."
2,Station EVA 8000105,3704713885565394549-2410241036-11,NaN,NaN,NaN,NaN,2.410241e+09,NaN,NaN,NaN,"[{'message_id': 'r8523678', 'type': 'q', 'code..."
3,Station EVA 8000105,-8065221203242695110-2410241304-9,NaN,NaN,2.410241e+09,NaN,NaN,RB58,NaN,NaN,No messages
4,Station EVA 8000105,5656583513138578556-2410241010-10,NaN,NaN,2.410241e+09,NaN,NaN,NaN,NaN,NaN,"[{'message_id': 'r8508380', 'type': 'f', 'code..."
